In [1]:
import itertools
import numpy as np

from scipy.sparse import csr_matrix

from utilities import chol_params_to_lower_triangular_matrix
from utilities import cov_matrix_to_sdcorr_params
from utilities import number_of_triangular_elements_to_dimension
from utilities import dimension_to_number_of_triangular_elements

from utilities import commutation_matrix
from utilities import elimination_matrix
from utilities import duplication_matrix

from jax import jacfwd
from kernel_transformations_jax import covariance_from_internal as covariance_from_internal_jax
from kernel_transformations_jax import sdcorr_from_internal as sdcorr_from_internal_jax
from kernel_transformations_jax import probability_from_internal as probability_from_internal_jax

from numpy.testing import assert_array_almost_equal

$$
\tilde{\text{vec}}
\left (
\begin{matrix}
(0,0)  &        &        &        \\
(1, 0) & (1,1)  &        &        \\
(2, 0) & (2, 1) & (2, 2) &        \\
(3, 0) & (3, 1) & (3, 2) & (3, 3) \\
\end{matrix}
\right ) =:
\tilde{\text{vec}}(L) = 
\big ( (0,0), (1,0), (1,1), (2,0), (2, 1), (2, 2), (3, 0), (3, 1), (3, 2), (3, 3) \big )^\top := v
$$

The following two functions allow us to move between these two representation of a (lower-triangular) matrix in a bijective fashion.

In [2]:
MAX_VALUE = 500

SEQUENCE_I = list(itertools.chain.from_iterable(itertools.repeat(i-1, i) for i in range(1, MAX_VALUE)))
SEQUENCE_J = list(itertools.chain.from_iterable(range(i-1) for i in range(1, MAX_VALUE)))

def _vectorized_index_to_matrix_index(index):
    return SEQUENCE_I[index], SEQUENCE_J[index]

def _matrix_index_to_vectorized_index(i, j):
    return int(i * (i + 1) / 2) +  j

for k in range(100):
    assert _matrix_index_to_vectorized_index(*_vectorized_index_to_matrix_index(k)) == k

In [3]:
def seq(n_list, max_n=10):
    n = len(n_list)
    if n < max_n:
        n_list.append(n_list[-1] + n + 1)
        n_list = seq(n_list, max_n)
        
    return n_list


TO_DROP_SEQUENCE = seq([0], max_n=500)
SEQUENCE_SDCORR = [e for e in range(125250) if e not in TO_DROP_SEQUENCE]

## Derivative of ``covariance_from_internal``

The graph we want to differentiate looks as follow

$$\tilde{\text{vec}}(L) \to L \to L L^\top =: \Sigma \to \tilde{\text{vec}}(\Sigma) \,,$$
where $L$ is the Cholesky factor of the covariance matrix $\Sigma$.
Let us define a function $f: \mathbb{R}^m \to \mathbb{R}^m, x \mapsto f(x)$, which takes an internal vector $x$ (of correct dimension) and transforms it to the covariance matrix as depicted above.
We want to find the Jacobian of $f$, i.e.
$$
J(f) = \left ( \frac{\partial \, f_i}{\partial \, x_j} \right )_{i, j = 1, \dots, m}
$$

We tackle this problem by finding an explicit expression for ${\partial f_i}/{\partial x_j}$ and then looping over $i,j = 1,\dots, m$.

Henceforth let $i, j$ be given and let $\Sigma = (\sigma_{i, j})$.
Note that for each $j$ we can find a unique index tuple $(a, b) = (a(j), b(j))$ such that $\tilde{\text{vec}}(L)_j = L_{a, b}$ and equivalently we find $(n, m) = (n(i), m(i))$ such that $\tilde{\text{vec}}(\Sigma)_i = \sigma_{n, m}$. Also note that for these indices it always holds that $a \geq b$ and $n \geq m$.

Now note again that with $L = \left [ \begin{matrix} \ell_1 \\ \vdots \\ \ell_m \end{matrix} \right ]$, we have $\sigma_{k, l} = \ell_k^ \, \bullet \ell_l$.
Hence we get

$$
\frac{\partial \, f_i}{\partial \, x_j} = 
\frac{\partial}{\partial L_{a, b}} \left ( \ell_n \bullet \, \ell_m \right ) = 
\frac{\partial}{\partial L_{a, b}} \left ( \sum_{k=1}^{m} L_{n, k} L_{m, k} \right ) = 
\mathbb{1}(b \leq m) \left [ \mathbb{1}(a = n) L_{m, b} + \mathbb{1}(a = m) L_{n, b} \right ]
$$

In [12]:
def derivative_covariance_from_internal(internal_values):
    chol = chol_params_to_lower_triangular_matrix(internal_values)
    
    dim = len(chol)
    
    K = commutation_matrix(dim)
    
    d0_a = np.eye(dim ** 2) + K
    d0_b = np.kron(chol, np.eye(dim))
    d0 = d0_a @ d0_b
    
    L = elimination_matrix(dim)
    D = duplication_matrix(dim, L)
    
    d1 = L @ d0 @ D
    
    return d1

## Example / Testing

In [13]:
J = jacfwd(covariance_from_internal_jax)

In [2]:
def get_random_internal(dim, seed=0):
    np.random.seed(seed)
    chol = np.tril(np.random.randn(dim, dim))
    internal = chol[np.tril_indices(len(chol))]
    return internal

In [15]:
for dim in range(10, 50):
    internal = get_random_internal(dim)

    jax_deriv = J(internal)

    my_deriv = derivative_covariance_from_internal(internal)

    assert_array_almost_equal(jax_deriv, my_deriv)

## Timeit

In [16]:
internal = get_random_internal(20)

In [17]:
%timeit J(internal)

13.4 ms ± 187 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%timeit derivative_covariance_from_internal(internal)

15.7 ms ± 332 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Derivative of ``sdcorr_from_internal``

The graph we want to differentiate looks as follow

$$\tilde{\text{vec}}(L) \to L \to L L^\top =: \Sigma \to \mathcal{P} \to \bar{\text{vec}}(\mathcal{P}) \,,$$
where $L$ is the Cholesky factor of the covariance matrix $\Sigma$ and $\mathcal{P}$ denotes the *modified* correlation matrix. With modified correlation matrix we mean a correlation matrix that has the standard deviations written on the diagonal instead of ones. Moreover, the new operate $\bar{\text{vec}}$ maps the diagonal elements of a matrix to the first entries of the resulting vector and proceeds to fill the remaining entries using the operator $\tilde{\text{vec}}$. That is, if we let $M$ be some lower-triangular, $m \times m$ matrix and define $M'$ as the $(m-1) \times (m-1)$ lower-triangular matrix containing the lower-triangular elements of $M$ except for the diagonal elements, then

$$ \bar{\text{vec}}(M) = \left ( \text{diag}(M), \tilde{\text{vec}}(M') \right )$$


Let us define a function $f: \mathbb{R}^m \to \mathbb{R}^m, x \mapsto f(x)$, which takes an internal vector $x$ (of correct dimension) and transforms it to the correlation matrix as depicted above.
We want to find the Jacobian of $f$, i.e.
$$
J(f) = \left ( \frac{\partial \, f_i}{\partial \, x_j} \right )_{i, j = 1, \dots, m}
$$

We tackle this problem by finding an explicit expression for ${\partial f_i}/{\partial x_j}$ and then looping over $i,j = 1,\dots, m$.

Henceforth let $i, j$ be given and let $\Sigma = (\sigma_{i, j})$ as well as $\mathcal{P} = (\rho_{i,j})$.
Note that for each $j$ we can find a unique index tuple $(a, b) = (a(j), b(j))$ such that $\tilde{\text{vec}}(L)_j = L_{a, b}$ and equivalently we find $(n, m) = (n(i), m(i))$ such that $\bar{\text{vec}}(\mathcal{P})_i = \rho_{n, m}$. Also note that for these indices it always holds that $a \geq b$ and $n \geq m$.

Now note again that with $L = \left [ \begin{matrix} \ell_1 \\ \vdots \\ \ell_m \end{matrix} \right ]$, we have $\sigma_{k, l} = \ell_k^ \, \bullet \ell_l$.

But by definition of a correlation matrix we thus have

$$
\rho_{n, m} = 
\frac{\sigma_{n, m}}{\sqrt{\sigma_{n, n}} \sqrt{\sigma_{m, m}}} =
\frac{\ell_n \, \bullet \ell_m}{||\ell_n||_2 \, ||\ell_m||_2} \,.
$$

Hence,

$$
\frac{\partial \, f_i}{\partial \, x_j} = 
\frac{\partial}{\partial L_{a(j), b(j)}} \rho_{n(i), m(i)} =
\frac{\partial}{\partial L_{a, b}} \left ( \frac{\ell_n \, \bullet \ell_m}{||\ell_n||_2 \, ||\ell_m||_2}
\right ) =: 
(\star)$$

To solve for $(\star)$ let us consider first

$$
(\star \star) :=
\frac{\partial}{\partial L_{a, b}} \left ( \ell_n \, \bullet \ell_m \right ) = 
\frac{\partial}{\partial L_{a, b}} \left ( \sum_{k=1}^{m} L_{n,k} L_{m, k} \right ) = 
\mathbb{1}(b \leq m, a = n) L_{m, b} + \mathbb{1}(b \leq m, a = m) L_{n, b}
$$

and 

$$
(\bullet) :=
\frac{\partial}{\partial L_{a, b}} ||\ell_n||_2 = \frac{\partial}{\partial L_{a, b}} \sqrt{\sum_{k=1}^n L_{n, k}^2} = \mathbb{1}(b \leq n, a = n) L_{n, b} ||\ell_n||_2^{-1}
$$


\begin{align}
(\bullet \, \bullet) :=
\frac{\partial}{\partial L_{a, b}} \left ( ||\ell_n||_2 \, ||\ell_m||_2 \right ) &= 
\frac{\partial}{\partial L_{a, b}} \left ( ||\ell_n||_2 \right ) ||\ell_m||_2 + 
\frac{\partial}{\partial L_{a, b}} \left ( ||\ell_m||_2 \right ) ||\ell_n||_2 \\
&= \mathbb{1}(b \leq n, a = n) L_{n, b} \frac{||\ell_m||_2}{||\ell_n||_2} + 
\mathbb{1}(b \leq m, a = m) L_{m, b}  \frac{||\ell_n||_2}{||\ell_m||_2}
\end{align}

Then, with $\alpha_{n, m} := ||\ell_n||_2 \, ||\ell_m||_2$, we get by applying the quotient rule

$$
(\star) = \frac{1}{\alpha_{n, m}^2} \left ( \alpha_{n, m} \times (\star \star) - \ell_n \, \bullet \ell_m \times (\bullet \, \bullet) \right )
$$

In [87]:
def _sdcorr_index(index, dim):
    if index < dim:
        i, j = index, index
    else:
        idx = SEQUENCE_SDCORR[index - dim]
        i, j = _vectorized_index_to_matrix_index(idx)
    return i, j

In [88]:
def derivative_sdcorr_from_internal(internal_values):
    dim = len(internal_values)
    
    chol = chol_params_to_lower_triangular_matrix(internal_values)
    n_chol = len(chol)
    
    deriv = np.zeros((dim, dim))
    for i in range(dim):
        
        n, m = _sdcorr_index(i, n_chol)
        
        for j in range(dim):
            
            a, b = _vectorized_index_to_matrix_index(j)
            
            if i < n_chol:
                deriv[i, j] = _derivative_sdcorr_from_internal_inner_sd(n, m, a, b, chol)
            else:
                deriv[i, j] = _derivative_sdcorr_from_internal_inner_corr(n, m, a, b, chol)

    return deriv

In [89]:
def _derivative_sdcorr_from_internal_inner_sd(n, m, a, b, chol): 
    if b <= n and a == n:
        ln_norm = np.sqrt(np.sum(chol[n] ** 2))
        deriv = chol[n, b] / ln_norm
    else:
        deriv = 0
    
    return deriv

In [90]:
def _derivative_sdcorr_from_internal_inner_corr(n, m, a, b, chol):
    ln_norm = np.sqrt(np.sum(chol[n] ** 2))
    lm_norm = np.sqrt(np.sum(chol[m] ** 2))

    alpha = ln_norm * lm_norm

    # \ell_n \bullet \ell_m
    dotprod = np.dot(chol[n], chol[m])


    # (\star \star)
    left = 0
    if b <= m:
        if a == n:
            left += chol[m, b]
        if a == m:
            left += chol[n, b]

    # (\bullet \bullet)
    right = 0
    if b <= n and a == n:
        right += chol[n, b] * lm_norm / ln_norm
    if b <= m and a == m:
        right += chol[m, b] * ln_norm / lm_norm

    deriv = (alpha * left - dotprod * right) / (alpha ** 2)

    return deriv

## Example / Testing

In [91]:
J = jacfwd(sdcorr_from_internal_jax)

In [102]:
for dim in range(10, 50):
    internal = get_random_internal(dim)
    jax_deriv = J(internal)
    my_deriv = derivative_sdcorr_from_internal(internal)
    assert_array_almost_equal(jax_deriv, my_deriv)
    bad.append(dim)

## Timeit

In [103]:
internal = get_random_internal(20)

In [104]:
%timeit J(internal)

23.5 ms ± 162 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [105]:
%timeit derivative_sdcorr_from_internal(internal)

1.37 s ± 28.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Derivative of ``probability_from_internal``

Let $f: \mathbb{R}^m \to \mathbb{R}^m, x \mapsto \frac{1}{x^\top 1} x$, with $1$ denoting a vector of all ones. Define $\sigma := x^\top 1 = \sum_k x_k$. Then,
$$
J(f)(x) = \frac{1}{\sigma} I_m - \frac{1}{\sigma^2} 1 x^\top \,,
$$
where $I_m$ denotes the $m \times m$ identity matrix.

In [36]:
def derivative_probability_from_internal(internal_values):
    dim = len(internal_values)
    
    sigma = np.sum(internal_values)
    
    left = np.eye(dim)
    
    right = np.ones((dim, dim)) * (internal_values / sigma)
    
    deriv = left - right.T
    deriv /= sigma
    return deriv

## Example / Testing

In [37]:
J = jacfwd(probability_from_internal_jax)

In [50]:
bad = []
for dim in range(5, 50):
    try:
        internal = get_random_internal(dim)
        jax_deriv = J(internal)
        my_deriv = derivative_probability_from_internal(internal)
        assert_array_almost_equal(jax_deriv, my_deriv, decimal=5)
    except AssertionError:
        bad.append(dim)
        
print(bad)

[16, 17]


## Timeit

In [22]:
internal = get_random_internal(20)

In [23]:
%timeit J(internal)

3.78 ms ± 42 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%timeit derivative_probability_from_internal(internal)

741 µs ± 14.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
